## Data Preparation

In [2]:
# import libraries
import datetime as dt
import os
import re
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# set data path
PATH = os.path.join('..', 'data/datasets')

### Filenames. "Hash" to Real Filename

Many table names use a hash instead of a real name, but there is a file available with the "decodings" (*names_mapping.csv*). Let's replace the hashed names with their real filenames.

In [4]:
for subdir in os.listdir(PATH):
   subdir_path = os.path.join(PATH, subdir)
   if os.path.isdir(subdir_path):
       
       # check if the mapping file exists in the current subdirectory
       mapping_file = os.path.join(subdir_path, 'names_mapping.csv')
       print(subdir)
       if os.path.exists(mapping_file):
           mapping_df = pd.read_csv(mapping_file)
           hash_to_name = dict(zip(mapping_df['hash'], mapping_df['table_name']))

           for filename in os.listdir(subdir_path):
               file_base_name = os.path.splitext(filename)[0]         # strip the .csv extension
               if file_base_name in hash_to_name: 
                   file_path = os.path.join(subdir_path, filename)
                   real_name = hash_to_name[file_base_name] + '.csv'  # add the .csv extension to the real name
                   new_file_path = os.path.join(subdir_path, real_name)
                   os.rename(file_path, new_file_path)
                   print(f'Renamed: {file_path} -> {new_file_path}')
       else:
           print('Mapping file does not exist')

### Change Filenames

In the data from different years, there are tables covering the same topics, but their names may vary slightly. By using the table names from the most recent year as a reference, we will rename all the tables, keeping only their numbers. This will make it easier to navigate the data in the future.

In [5]:
# regular expression pattern for matching filenames
# capture one digit at the start of the string before the period and 1 or 2 digits after the period

pattern = re.compile(r'^(\d)\.(\d{1,2})') 

for subdir in os.listdir(PATH):
   subdir_path = os.path.join(PATH, subdir)
   if os.path.isdir(subdir_path):
       for filename in os.listdir(subdir_path):
           # check and rename files matching the pattern
           match = pattern.match(filename)
           if match:
               file_path = os.path.join(subdir_path, filename)
               new_name = f"{match.group(1)}_{match.group(2)}.csv"
               new_file_path = os.path.join(subdir_path, new_name)
               os.rename(file_path, new_file_path)

### Merge files with two parts
Some tables consisted of two parts, which were merged before loading the data.

In [ ]:
def concat_tables(year, filename1='2_1_reg_1.csv', filename2='2_1_reg_2.csv', newfilname='2_1_reg.csv'):
    df_1 = pd.read_csv(os.path.join(*[PATH], year, filename1), header = [1], encoding='utf-8')
    df_2 = pd.read_csv(os.path.join(*[PATH], year, filename2), header = [1], encoding='utf-8')
    df = pd.concat([df_1, df_2.iloc[:, 1:]], axis=1)
    path = os.path.join(*[PATH], year, newfilname)
    return df.to_csv(path, index=False)

In [ ]:
for year in ['2001', '2007', '2009', '2011']:
    print('\033[1m'+year+'\033[0m')
    print('___________________')

    if year == '2001':
        concat_tables(year)
        print(f'File created successfully')
        concat_tables(year, filename1='2_1_reg_1995,1998_1.csv', filename2='2_1_reg_1995,1998_2.csv', newfilname='2_1_reg_old.csv')
        print(f'File for 1995 created successfully')
    else:
        concat_tables(year)
        print(f'File created successfully')

## Conclusion
Not all tables from previous years (mainly before 2013) have the same numbers for the same names as in the most recent version. Tables without numbers, with mismatched numbers, and those with non-renamed hashed names (in some cases, the mapping file was empty but hashed names were present; sometimes, the mapping file name itself was a hash) were renamed manually. Not all years contain the same set of tables; for example, after 2013, there are no tables with regional data.